In [324]:
import pymysql
from pykrx import stock
import pandas as pd
import time


def db_connect():
    db = pymysql.connect(host='localhost',
                         port=3306,
                         user='root',
                         passwd='1234',
                         db='stockdb',
                         charset='utf8')
    return db

def sise(start_date, end_date):
    err_list = []
    try:
        for i in stock.get_market_ticker_list():
            df = stock.get_market_ohlcv_by_date(str(start_date), str(end_date), i)
            df['종목명'] = stock.get_market_ticker_name(i)
            df['종목코드'] = i
            df.reset_index(inplace = True)
            df.columns = ['stock_date', 'open_price', 'high', 'low', 'close', 'volume', 'stock_name', 'stock_code']
            df = df[['stock_date', 'stock_code', 'stock_name', 'open_price', 'high', 'low', 'close', 'volume']]
    
            if (i == stock.get_market_ticker_list()[0]):
                data = df
            else:
                data2 = df
                data = pd.concat([data, data2])
    except:
        err_list.append(i)
    return data

In [325]:
st_time = time.time()
db = db_connect() 
cursor = db.cursor() 

print('검색시작 일자를 입력하세요 (YYYYMMDD)')
start_date = input()
print('검색마지막 일자를 입력하세요 (YYYYMMDD)')
end_date = input()
#print('검색할 종목 개수를 입력하세요')
#cnt = int(input())
print('(해당 일자에 상장이 되지 않았거나 상장폐지된 경우 그 내용 빠짐)')

data = sise(start_date, end_date)

for row in range(len(data)):
    insert_data = list(data.iloc[row].values)
    sql = """
    insert into ohlcv IF NOT EXISTS values (%s, %s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(sql, (insert_data))
cursor.close()
db.commit()
end_time = time.time()

print('완료!')
print('소요시간 = ', end_time-st_time)


검색시작 일자를 입력하세요 (YYYYMMDD)
20200101
검색마지막 일자를 입력하세요 (YYYYMMDD)
20201231
(해당 일자에 상장이 되지 않았거나 상장폐지된 경우 그 내용 빠짐)
완료!
소요시간 =  614.7757670879364
